# Deploy and inference Jina Embeddings V3 with Azure Virtual machine

This notebook demonstrates how to deploy a [Jina Embeddings V3](https://jina.ai/news/jina-embeddings-v3-a-frontier-multilingual-embedding-model) model-powered [Azure Virtual machine](https://azuremarketplace.microsoft.com/en-us/marketplace/apps/jinaai.jina-embeddings-v3-vm?tab=Overview) and perform inference with the application within the virtual machine.

## Deploy the virtual machine

To deploy the virtual machine, start by consulting the [official deployment guide](https://learn.microsoft.com/en-us/azure/virtual-machines/windows/quick-create-portal). This document provides comprehensive steps for the deployment process.

It's worth mentioning that in the Basics tab of the VM setup, you will need to provide several details about the VM. 

You can customize the VM size used, and for certain sizes, you might need to adjust the allowed quota to ensure access. It is recommended to use the [Standard_NC4as_T4_v3](https://learn.microsoft.com/en-us/azure/virtual-machines/nct4-v3-series) VM. This VM features up to 1 NVIDIA T4 GPU with 16 GB of memory.

For the other tabs, you can leave most settings as default or adjust them to fit your needs.

<img src="images/deploy_embedding_v3_vm.png" width="50%" height="50%">

Once the deployment of the VM is complete, proceed to the resource group created for your deployment to verify the resources that have been established. 

Within the resource group, you'll find the public IP through which you can access your application within the VM.

Please note that the application within the VM will be unavailable immediately after deployment due to necessary model loading process **We recommend waiting at least 2 minutes before using the application.**

# Perform inference with the application

The Python example below demonstrates how to perform real-time inference using the public IP address of the deployed virtual machine.

In [ ]:
import json

import requests


def invoke_endpoint():
    url = "http://<Insert here your public IP address>/encode"  # For example: http://20.84.48.180/encode
    headers = {"Content-Type": "application/json"}
    json_data = {
        "data": [
            {
                "text": "Organic skincare for sensitive skin with aloe vera and chamomile: Imagine the soothing embrace of nature with our organic skincare range, crafted specifically for sensitive skin. Infused with the calming properties of aloe vera and chamomile, each product provides gentle nourishment and protection. Say goodbye to irritation and hello to a glowing, healthy complexion."
            },
            {
                "text": "Bio-Hautpflege für empfindliche Haut mit Aloe Vera und Kamille: Erleben Sie die wohltuende Wirkung unserer Bio-Hautpflege, speziell für empfindliche Haut entwickelt. Mit den beruhigenden Eigenschaften von Aloe Vera und Kamille pflegen und schützen unsere Produkte Ihre Haut auf natürliche Weise. Verabschieden Sie sich von Hautirritationen und genießen Sie einen strahlenden Teint."
            },
            {
                "text": "Cuidado de la piel orgánico para piel sensible con aloe vera y manzanilla: Descubre el poder de la naturaleza con nuestra línea de cuidado de la piel orgánico, diseñada especialmente para pieles sensibles. Enriquecidos con aloe vera y manzanilla, estos productos ofrecen una hidratación y protección suave. Despídete de las irritaciones y saluda a una piel radiante y saludable."
            },
            {
                "text": "针对敏感肌专门设计的天然有机护肤产品：体验由芦荟和洋甘菊提取物带来的自然呵护。我们的护肤产品特别为敏感肌设计，温和滋润，保护您的肌肤不受刺激。让您的肌肤告别不适，迎来健康光彩。"
            },
            {
                "text": "新しいメイクのトレンドは鮮やかな色と革新的な技術に焦点を当てています: 今シーズンのメイクアップトレンドは、大胆な色彩と革新的な技術に注目しています。ネオンアイライナーからホログラフィックハイライターまで、クリエイティビティを解き放ち、毎回ユニークなルックを演出しましょう。"
            }
        ],
        "parameters": {  # You can refer to https://jina.ai/news/jina-embeddings-v3-a-frontier-multilingual-embedding-model for details regarding the parameters.
            "task": "text-matching",  # Select the downstream task for which the embeddings will be used. The model will return the optimized embeddings for that task. Should be one of 'retrieval.query', 'retrieval.passage', 'separation', 'classification', 'text-matching' or None. None meaning no specific task is needed.
            "late_chunking": True,  # Apply the late chunking technique to leverage the model's long-context capabilities for generating contextual chunk embeddings.
            "dimensions": 512  # Output dimensions. Smaller dimensions are easier to store and retrieve, with minimal performance impact thanks to MRL. Must be in the range of 1 to 1024.
        } 
    }

    response = requests.post(url, headers=headers, data=json.dumps(json_data))
    print(response.json())


invoke_endpoint()